In [ ]:
import numpy as np
import cv2
import os
import sys
import insightface
from tqdm.auto import tqdm

real_samples_root = 'samples/'
synthetic_samples_root = 'results/'

real_features_root = 'features/real/'
synthetic_features_root = 'features/synthetic/'

if not os.path.exists(real_samples_root):
    sys.exit(f'Real samples root ({real_samples_root}) not found')
if not os.path.exists(synthetic_samples_root):
    sys.exit(f'Synthetic samples root ({synthetic_samples_root}) not found')

os.makedirs(real_features_root, exist_ok=True)
os.makedirs(synthetic_features_root, exist_ok=True)

overwrite = False

fr_model = insightface.app.FaceAnalysis(
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
ctx_id = 0
fr_model.prepare(ctx_id = ctx_id, det_thresh=0.1)

In [ ]:
print('Biometric feature extraction on real samples')
face_not_found = []
skipped = 0
for sample in tqdm(os.listdir(real_samples_root), file=sys.stdout):
    filename, extension = os.path.splitext(sample)
    sample_path = os.path.join(real_samples_root, sample)
    feature_path = os.path.join(real_features_root, filename + '.npy')
    if os.path.isfile(feature_path) and not overwrite:
        skipped += 1
        continue
    if '.jpg' or '.png' or '.jpeg' in extension.lower():
        face = fr_model.get(cv2.imread(sample_path))
        if face:
            sample = face[0].normed_embedding
            np.save(feature_path, sample)
        else: # face not found
            face_not_found.append(sample_path)

print(f'Samples found: {len(os.listdir(real_samples_root))}')
if not overwrite:
    print(f'Samples skipped: {skipped}')

if len(face_not_found) != 0:
      print("Number of faces not detected in real samples:",
            len(face_not_found), file=sys.stderr)
      print(*face_not_found, sep="\n", file=sys.stderr)

In [ ]:
print('Biometric feature extraction on synthetic samples')
face_not_found = []
skipped = 0

for sample in tqdm(os.listdir(synthetic_samples_root), file=sys.stdout):
    filename, extension = os.path.splitext(sample)
    sample_path = os.path.join(synthetic_samples_root, sample)
    feature_path = os.path.join(synthetic_features_root, filename + '.npy')
    if os.path.isfile(feature_path) and not overwrite:
        skipped += 1
        continue
    if '.jpg' or '.png' or '.jpeg' in extension.lower():
        face = fr_model.get(cv2.imread(sample_path))
        if face:
            sample = face[0].normed_embedding
            np.save(feature_path, sample)
        else: # face not found
            face_not_found.append(sample_path)

print(f'Samples found: {len(os.listdir(synthetic_samples_root))}')
if not overwrite:
    print(f'Samples skipped: {skipped}')

if len(face_not_found) != 0:
      print("Number of faces not detected in real samples:",
            len(face_not_found), file=sys.stderr)
      print(*face_not_found, sep="\n", file=sys.stderr)